In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math as m

In [2]:
#defining entropy function for decision  tree
def entropy(y):
    set_val = set(y)
    total = len(y)
    single_entropy = 0
    for l in set_val:
        p = (y == l).sum() / total
        single_entropy += -p*m.log2(p)
    return single_entropy

In [3]:
# calculating information gain for decsion tree
def info_gain_modified(x , y , index):
    
    #taking out unique vals in y so as to calculate initial info_gain
    vals = set(y)
    info_orig = 0
    
    #calculating original info gain of parent node
    for f in vals :
        temp = y.loc[y == f]
        info_orig += (-1*len(temp)/len(y))*entropy(temp)
        
    #IN THE CODE , WE ARE BASICALLY TRYING TO IDENTIFY THE BEST FEATURE AMONGST MEAN VALUES , RATHER THAN TAKING OUT AVERAGE FOR 
    # ALL DATA.
    
    step = x[x.columns[index]].mean()
    init = step/2
    condition = 0
    maxm = -1e100
    split_info_max = -1e100
    
    # RUNNING LOOP FROM 0 TO MAX VALUE.
    while init <= 2*step :
        final_entropy = 0
        y_new_greater  = y.loc[x[x.columns[index]] >= init]
        y_new_less  = y.loc[x[x.columns[index]] < init]
        
        ######################################################## CALCULATING COEFF FOR INFO_GAIN.
        d1 = len(y_new_greater)
        d2 = len(y_new_less)
        d = d1 + d2
        D1 = float(-1*d1)/d
        D2 = float(-1*d2)/d
        #########################################################
        
        #INFORMATION REQUIRED CALCULATED :
        final_entropy = float(D1*(entropy(y_new_greater))) + float(D2*(entropy(y_new_less)))
        if final_entropy > maxm:
            maxm = final_entropy
            condition = init
            split_info_max = split_info(y_new_greater , y_new_less) 
        init += init
    #RETURING INFORMATION GAIN , CONDITION FOR SPLITTING , AND SPLIT INFO FOR OPTIMAL SPLIT.    
    return info_orig  - maxm, condition , split_info_max

In [4]:
#CALCULATING SPLIT INFO FOR A CLASS
def split_info(y1 , y2) :
    d1 = len(y1)
    d2 = len(y2)
    d = d1 + d2
    if(d1 == 0 or d2 == 0 or d == 0):
        return 0
    p = (-1*d1*m.log2(d1/d))/d +  (-1*d2*m.log2(d2/d))/d
    return p

In [5]:
#CALCULATING GAIN RATIO
def gain_ratio(x ,y ,index):
    return info_gain_modified(x , y , index)[0] / info_gain_modified(x , y , index)[2]

In [6]:
def dt_modified(x , y , feature_index , level):
    
    #SHOWS HOW MANY MORE FEATURES ARE LEFT TO SPLIT UPON
    print('-------------------------------------------------------------------------------------------')
    print('features list left to split upon : ' , feature_index)
    
    #BASE CASE : pure node : return
    if(len(set(y)) == 1): 
        print('Level : ' , level)
        print('Current entropy : ' , 0.0)
        print('count of feature :' , len(y))
        print('Reached leaf node')
        print('-------------------------------------------------------------------------------------------')
        print()
        return
    
    #BASE CASE : all features are exhausted
    if(len(feature_index) == 0):
        print('Level : ' , level)
        print('Current entropy : ' , entropy(y))
        print('number of features : ' , len(y))
        print('leaf node reached , no features left')
        print('-------------------------------------------------------------------------------------------')
        print()
        return
    
    ################################################################################
    maxm = -10000000
    best_feature = -1
    condition = 0
    for i in feature_index:
        gain = gain_ratio(x , y , i)
        if gain > maxm:
            maxm = gain
            best_feature = i
            condition = info_gain_modified(x , y , i)[1]
            
    # printing required data:
    
    print('Level : ' , level)
    print('Current entropy : ' , entropy(y))
    print('Splitting on feature' , x.columns[best_feature] , 'on criteria that data is centered: ' , condition  
          , 'with gain ratio : ' , maxm)
    print('num_features :' , len(y))
    
    # finding all classes to split upon
    labels = set(x[x.columns[best_feature]])
    
    #removing the used feature
    if best_feature not in feature_index:
        return
    feature_index.remove(best_feature)
    
    x_new_greater  = x.loc[x[x.columns[best_feature]] >= condition]
    y_new_greater  = y.loc[x[x.columns[best_feature]] >=  condition]
    x_new_lesser  = x.loc[x[x.columns[best_feature]] < condition]
    y_new_lesser  = y.loc[x[x.columns[best_feature]] <  condition]
    
    print('Number of features which are > ' , condition , ' : ' , len(y_new_greater))
    print('Number of features which are < ' ,   condition , ' : ' , len(y_new_lesser))
    print('-------------------------------------------------------------------------------------------')
    print()
    
    #recursively calling DT to print features
    dt_modified(x_new_greater , y_new_greater , feature_index , level + 1)
    dt_modified(x_new_lesser , y_new_lesser , feature_index , level + 1)
    feature_index.append(best_feature)

In [7]:
iris = datasets.load_iris()

#  converting dataset to pandas DataFrame
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
df.columns = ['sl' , 'sw' , 'pl' , 'pw' , 'target']

#  dataset splitting into train and test
x_train , x_test = train_test_split(df.iloc[: , 0:-1] , random_state = 20)
y_train , y_test = train_test_split(df.iloc[: , -1] , random_state = 20)
list_of_features = x_train.columns.tolist()

#testing some points :
#x_train[x_train.columns[1]]
#new = y_train.loc[x_train[x_train.columns[0]] == 6.4]

#taking out feature indices(column indices)
feature_idx = [i for i in range(len(x_train.columns))]
feature_idx

[0, 1, 2, 3]

In [8]:
#CALLING THE DECISION TREE FUNCTION TO PRINT THE REQUIRED TREE
feature_idx = [i for i in range(len(x_train.columns))]
dt_modified(x_train , y_train , feature_idx , 0)

-------------------------------------------------------------------------------------------
features list left to split upon :  [0, 1, 2, 3]
Level :  0
Current entropy :  1.5844996446144277
Splitting on feature sw on criteria that data is centered:  3.088392857142856 with gain ratio :  1.3177216932877882
num_features : 112
Number of features which are >  3.088392857142856  :  55
Number of features which are <  3.088392857142856  :  57
-------------------------------------------------------------------------------------------

-------------------------------------------------------------------------------------------
features list left to split upon :  [0, 2, 3]
Level :  1
Current entropy :  1.3090909691036434
Splitting on feature sl on criteria that data is centered:  5.74181818181818 with gain ratio :  0.4548646284299678
num_features : 55
Number of features which are >  5.74181818181818  :  23
Number of features which are <  5.74181818181818  :  32
------------------------------------